In [1]:
import pandas as pd 
import numpy as np
import aquire as aq
import prepare as pr
import warnings 
warnings.filterwarnings('ignore')

import graphviz
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [2]:
#get and prepare titanic data
t_df = aq.new_titanic_data()
t_df[:1]

,Unnamed: 0,passenger_id,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,deck,embark_town,alone
0,0,0,0,3,male,22.0,1,0,7.25,S,Third,NaN,Southampton,0


In [3]:
#prep and create our train, validate and test dataframes/ creates dummies for string catagoricals
train, validate, test = pr.prep_titanic(t_df)
train[:1]

,survived,pclass,age,sibsp,parch,fare,alone,Q,S,male
583,0,1,36.0,0,0,40.125,1,0,0,1


In [10]:
#create x and y values for train x = df with the target missing, y = target variable
X_train = train.drop(columns=['survived']) 
y_train = train.survived
X_validate = validate.drop(columns=['survived']) 
y_validate = validate.survived
X_test = test.drop(columns=['survived']) 
y_test = test.survived


In [68]:
#1
#What is your baseline prediction? What is your baseline accuracy? 
#remember: your baseline prediction for a classification problem is predicting 
#the most prevelant class in the training dataset (the mode). 
#When you make those predictions, what is your accuracy? This is your baseline accuracy.

In [69]:
#find best base case
train.survived.value_counts()

0    307
1    190
Name: survived, dtype: int64

In [70]:
#set base case to 0
base_model = DummyClassifier(strategy='constant', constant=0)

In [71]:
#fit our model
base_model.fit(X_train, y_train)

DummyClassifier(constant=0, strategy='constant')

In [96]:
baseline = base_model.score(X_train , y_train)
print('Baseline accuracy: {:.3f} %'.format(baseline * 100))

Baseline accuracy: 61.771 %


In [74]:
#2 Fit the decision tree classifier to your 
#training sample and transform (i.e. make predictions on the training sample)
train[:9]

,survived,pclass,age,sibsp,parch,fare,alone,Q,S,male,pred,pred_1,pred_2,pred_3
583,0,1,36.000000,0,0,40.1250,1,0,0,1,0,0,0,0
337,1,1,41.000000,0,0,134.5000,1,0,0,0,1,1,1,1
50,0,3,7.000000,4,1,39.6875,0,0,1,1,0,0,0,0
218,1,1,32.000000,0,0,76.2917,1,0,0,0,1,1,1,1
31,1,1,29.916875,1,0,146.5208,0,0,0,0,1,1,1,1
308,0,2,30.000000,1,0,24.0000,0,0,0,1,0,0,0,0
314,0,2,43.000000,1,1,26.2500,0,0,1,1,0,0,0,0
883,0,2,28.000000,0,0,10.5000,1,0,1,1,0,0,0,0
459,0,3,29.916875,0,0,7.7500,1,1,0,1,0,0,0,0


In [84]:
#create our trees
clf_1 = DecisionTreeClassifier(max_depth=4)
clf_2 = DecisionTreeClassifier(max_depth=2, random_state=123)
clf_3 = DecisionTreeClassifier(max_depth=5, random_state=980)

In [85]:
#train our trees
clf_1.fit(X_train, y_train)
clf_2.fit(X_train, y_train)
clf_3.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, random_state=980)

In [121]:
train['pred_1'] = clf_1.predict(X_train)
train['pred_2'] = clf_2.predict(X_train)
train['pred_3'] = clf_3.predict(X_train)

In [122]:
#visualise data

In [123]:
dot_data_1 = export_graphviz(clf_1, feature_names= X_train.columns ,rounded=True, filled=True)
dot_data_2 = export_graphviz(clf_2, feature_names= X_train.columns ,rounded=True, filled=True)
dot_data_3 = export_graphviz(clf_3, feature_names= X_train.columns ,rounded=True, filled=True)
graph_1 = graphviz.Source(dot_data_1)
graph_2 = graphviz.Source(dot_data_2)
graph_3 = graphviz.Source(dot_data_3)
graph_1.render('titanic_tree_Model_1', view=True)
graph_2.render('titanic_tree_Model_2', view=True)
graph_3.render('titanic_tree_Model_3', view=True)

'titanic_tree_Model_3.pdf'

In [124]:
model_1_score = clf_1.score(X_train,y_train)
model_2_score = clf_2.score(X_train,y_train)
model_3_score = clf_3.score(X_train,y_train)

validate_model_1 = clf_1.score(X_validate,y_validate)
validate_model_2 = clf_2.score(X_validate,y_validate)
validate_model_3 = clf_3.score(X_validate,y_validate)

print('Model 1 accuracy: {:.3f} % | Model 1 accuracy on validate: {:.3f} %'.format(model_1_score * 100 ,validate_model_1 * 100))
print('Model 2 accuracy: {:.3f} % | Model 2 accuracy on validate: {:.3f} %'.format(model_2_score * 100 ,validate_model_2 * 100))
print('Model 3 accuracy: {:.3f} % | Model 3 accuracy on validate: {:.3f} %'.format(model_3_score * 100 ,validate_model_3 * 100))

Model 1 accuracy: 83.501 % | Model 1 accuracy on validate: 78.505 %
Model 2 accuracy: 79.074 % | Model 2 accuracy on validate: 78.037 %
Model 3 accuracy: 85.714 % | Model 3 accuracy on validate: 79.439 %


In [125]:
#in terms of training, accuracy, model 3 is the most accurate but has overfitting.
#model 2 preformed reletivly the same for the train and validate. use model 2

In [126]:
#compare validate to actual classification report for models
y_pred_1 = clf_1.predict(X_validate)
y_pred_2 = clf_2.predict(X_validate)
y_pred_3 = clf_3.predict(X_validate)
report_1 = classification_report(y_validate, y_pred_1,output_dict=True)
report_2 = classification_report(y_validate, y_pred_2,output_dict=True)
report_3 = classification_report(y_validate, y_pred_3,output_dict=True)

In [127]:
df_1, df_2 , df_3 = pd.DataFrame(report_1).transpose(), pd.DataFrame(report_2).transpose(), pd.DataFrame(report_3).transpose()

In [128]:
df_1, df_2, df_3

(              precision    recall  f1-score     support
 0              0.762195  0.946970  0.844595  132.000000
 1              0.860000  0.524390  0.651515   82.000000
 accuracy       0.785047  0.785047  0.785047    0.785047
 macro avg      0.811098  0.735680  0.748055  214.000000
 weighted avg   0.799672  0.785047  0.770611  214.000000,
               precision    recall  f1-score     support
 0              0.748538  0.969697  0.844884  132.000000
 1              0.906977  0.475610  0.624000   82.000000
 accuracy       0.780374  0.780374  0.780374    0.780374
 macro avg      0.827757  0.722653  0.734442  214.000000
 weighted avg   0.809248  0.780374  0.760247  214.000000,
               precision    recall  f1-score     support
 0              0.805556  0.878788  0.840580  132.000000
 1              0.771429  0.658537  0.710526   82.000000
 accuracy       0.794393  0.794393  0.794393    0.794393
 macro avg      0.788492  0.768662  0.775553  214.000000
 weighted avg   0.792479  0.7

In [131]:
#compare train to actual classification report for models
y_pred_1 = clf_1.predict(X_train)
y_pred_2 = clf_2.predict(X_train)
y_pred_3 = clf_3.predict(X_train)
report_1 = classification_report(y_train, y_pred_1,output_dict=True)
report_2 = classification_report(y_train, y_pred_2,output_dict=True)
report_3 = classification_report(y_train, y_pred_3,output_dict=True)
df_1, df_2 , df_3 = pd.DataFrame(report_1).transpose(), pd.DataFrame(report_2).transpose(), pd.DataFrame(report_3).transpose()

In [132]:
df_1, df_2, df_3

(              precision    recall  f1-score    support
 0              0.798408  0.980456  0.880117  307.00000
 1              0.950000  0.600000  0.735484  190.00000
 accuracy       0.835010  0.835010  0.835010    0.83501
 macro avg      0.874204  0.790228  0.807800  497.00000
 weighted avg   0.856361  0.835010  0.824825  497.00000,
               precision    recall  f1-score     support
 0              0.751861  0.986971  0.853521  307.000000
 1              0.957447  0.473684  0.633803  190.000000
 accuracy       0.790744  0.790744  0.790744    0.790744
 macro avg      0.854654  0.730327  0.743662  497.000000
 weighted avg   0.830455  0.790744  0.769524  497.000000,
               precision    recall  f1-score     support
 0              0.853293  0.928339  0.889236  307.000000
 1              0.865031  0.742105  0.798867  190.000000
 accuracy       0.857143  0.857143  0.857143    0.857143
 macro avg      0.859162  0.835222  0.844051  497.000000
 weighted avg   0.857780  0.857143 

In [112]:
#confusion matrix for models for train
labels = ["Dead", "Survived"]
y_pred_1 = clf_1.predict(X_train)
y_pred_2 = clf_2.predict(X_train)
y_pred_3 = clf_3.predict(X_train)

cm_1 = pd.DataFrame(confusion_matrix(y_train, y_pred_1),index=labels,columns=labels)
cm_2 = pd.DataFrame(confusion_matrix(y_train, y_pred_2),index=labels,columns=labels)
cm_3 = pd.DataFrame(confusion_matrix(y_train, y_pred_3),index=labels,columns=labels)

In [117]:
cm_1, cm_2, cm_3

(          Dead  Survived
 Dead       125         7
 Survived    39        43,
           Dead  Survived
 Dead       128         4
 Survived    43        39,
           Dead  Survived
 Dead       116        16
 Survived    28        54)

In [115]:
#confusion matrix for models for validate
labels = ["Dead", "Survived"]
y_pred_1 = clf_1.predict(X_validate)
y_pred_2 = clf_2.predict(X_validate)
y_pred_3 = clf_3.predict(X_validate)

cm_1 = pd.DataFrame(confusion_matrix(y_validate, y_pred_1),index=labels,columns=labels)
cm_2 = pd.DataFrame(confusion_matrix(y_validate, y_pred_2),index=labels,columns=labels)
cm_3 = pd.DataFrame(confusion_matrix(y_validate, y_pred_3),index=labels,columns=labels)

In [116]:
cm_1, cm_2, cm_3

(          Dead  Survived
 Dead       125         7
 Survived    39        43,
           Dead  Survived
 Dead       128         4
 Survived    43        39,
           Dead  Survived
 Dead       116        16
 Survived    28        54)

### #6 Which model performs better on your in-sample data?
#### for Positive = Dead

- Best for accuracy: Model 3
- Best for Recall : Model 2
- Best for Precision: Model  3


### #7 Which model performs better on your out-sample data?
#### for Positive = Dead

- Best for accuracy: Model 3
- Best for Recall : Model 2
- Best for Precision: Model  3